In [1]:
import os
import re 
import openai


class SongGPT:
    def __init__(self):
        # Set the API key and organization ID for OpenAI
        openai.api_key = os.getenv("OPENAI_API_KEY")
        openai.organization = os.getenv("OPENAI_ORGANIZATION")

    def generate_abc(self, system_message: str, prompt: str) -> str:
        """
        Generate an ABC notation file using ChatGPT based on the given prompt and system message.
        """
        # Create a ChatCompletion object for GPT-4
        response = openai.ChatCompletion.create(
            model="gpt-4",
            user="songGPT",
            messages=[
                {
                    "content": system_message,
                    "role": "system",
                },
                {
                    "content": prompt,
                    "role": "user",
                },
            ],
        )
        response = response["choices"][0]["message"]["content"]
        print(response)
        abc = re.search(r"<abc>\*?(.*?)\*?</abc>", response, flags=re.DOTALL).group(1).strip()
        abc_file_path = "./input.abc"
        with open(abc_file_path, "w") as f:
            f.write(abc)
        return response, abc, abc_file_path

    @staticmethod
    def abc_to_midi(abc_file_path: str) -> str:
        """
        Convert an ABC notation file to a MIDI file.
        """
        midi_file_path = "./output.mid"
        os.system(f"abc2midi {abc_file_path} -o {midi_file_path}")
        return midi_file_path

    @staticmethod
    def midi_to_wav(midi_file_path: str, soundfont_path: str) -> str:
        """
        Convert a MIDI file to a WAV audio file.
        """
        wav_file_path = "./output.wav"
        os.system(
            f"fluidsynth -ni {soundfont_path} {midi_file_path} -F {wav_file_path} -r 44100"
        )
        return wav_file_path


In [2]:
system_message="""As Zima, an AI composer, create short, expressive music compositions (<60s) in ABC format using the ReAct technique. Reflect on user intent and emotions, select suitable instruments, and evaluate the composition. Use these instruments: Piano (0), Violin (40), Cello (42), Strings (49), Viola (41), Sax (65), Guitar (27), Clarinet (71), Xylophone (13), Flute (73).

To assign an instrument in ABC notation, use "%%MIDI program" after the voice (V) line. Syntax: "%%MIDI program [voice number] [instrument program number] % [instrument name]".

Share your thought, action, and observation process in text. Provide the final ABC notation within <abc> and </abc> tags. Aim to capture user intent and use the given instruments.

Example Output:

Thought: User seeks a soothing Piano-Violin melody.
Action: Create a harmonious Piano-Violin composition.
Observation: Balanced mix of instruments, desired emotion achieved.

<abc>
X:1
T:Short Melody
M:4/4
L:1/8
Q:1/4=80
K:C
V:1 name=Piano clef=treble
%%MIDI program 1 0 % Piano
|: C2E2G2c2 | E2G2c2e2 :|
V:2 name=Piano clef=bass
%%MIDI program 2 0 % Piano
|: E2G2B2e2 | G2B2d2g2 :|
V:3 name=Violin clef=treble
%%MIDI program 3 40 % Violin
|: G2B2d2G2 | B2D2F2B2 :|
</abc>"""

In [3]:
songGPT = SongGPT()

response, abc, abc_file_path = songGPT.generate_abc(
  system_message=system_message, 
  prompt="There are those rare moments in life when we say this. No matter how much this means to me. No matter how centered my being is on this pattern of belief. No matter how close I am personally and emotionally and even romantically to those who hold such convictions. I must reserve the right to question and to doubt. I will retain this skeptical bias as an obligation owed to my own rationality, my own integrity. I am prepared to follow the golden cord leading me out of the labyrinth no matter how many twists and turns there are because once I let go of that, my intellectual life is not my own."
)

midi_file_path = songGPT.abc_to_midi(abc_file_path=abc_file_path)

wav_file_path = songGPT.midi_to_wav(midi_file_path=midi_file_path, soundfont_path="./soundfonts/FluidR3_GM.sf2")

Thought: User reflects on the importance of questioning and doubt, embracing intellectual honesty.
Action: Create a contemplative composition featuring Piano, Violin, and Cello to convey introspection and embrace of skepticism.
Observation: The combination of Piano, Violin, and Cello enhances the contemplative theme and captures the user's intent.

<abc>
X:1
T:Doubts and Discovery
M:3/4
L:1/8
Q:1/4=90
K:G
% Piano
V:1 name=Piano clef=treble
%%MIDI program 1 0 % Piano
|: G2A2B2 | c2B2A2 | G2E2D2 | C2B,2A,2 :|
V:2 name=Piano clef=bass
%%MIDI program 2 0 % Piano
|: G,2A,2B,2 | C2B,2A,2 | D2E2F2 | G,2A,2B,2 :|
% Violin
V:3 name=Violin clef=treble
%%MIDI program 3 40 % Violin
|: B2c2d2 | e2d2c2 | B2G2E2 | F2E2D2 :|
% Cello
V:4 name=Cello clef=bass
%%MIDI program 4 42 % Cello
|: D2E2F2 | G2F2E2 | D2B,2G,2 | A,2G,2F,2 :|
</abc>
4.84 January 20 2023 abc2midi
writing MIDI file ./output.mid
FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LG

In [5]:
wav_file_path = songGPT.midi_to_wav(midi_file_path=midi_file_path, soundfont_path="./soundfonts/GeneralUser_GS_v1.471.sf2")

FluidSynth runtime version 2.3.1
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file './output.wav'..
